Para a raspagem do Mercado Livre, foi utilizado um código para a primeira página e copiado para gerar da segunda página em diante. 
Na verdade a estrutura do codigo são iguais a mudança ocorre na construção da url.

Assim, este notebook foi divido colocando um bloco de código para cada termo pesquisado (brinquedo, escritório..) e dentro deste bloco um bloco para a primeira página e outro para as páginas seguintes (página 2 em diante).

As funções de extrair primeira página e páginas seguintes não foram terminadas, o que deixaria o código mais enxuto e legível.

A função extrai_valor_parcelas esta em aplicação, ela é usada após a raspagem para tratar a coluna Parcelas que na extração possui a quantidade de parcelas e o valor juntos, esta função cria mais 2 colunas com estes respectivos valores.


## URL de pesquisa
A estrututa do url da primeira página é simples: composto pela url da homepage mais o item pesquisado conforme abaixo:

homepage  = "https://lista.mercadolivre.com.br/"
item pesquisa = 'brinquedo'
url da pesquisa = "{homepage}{item}#D[A:{item}]"

Exemplo: https://lista.mercadolivre.com.br/notebook#D[A:notebook]

Nas paginas seguintes a estrutura muda, é inserido na estrutura a categoria do item, sendo necessário pesquisar previamente, e o número da páginas dentro do url segue um adrão diferente para cada pesquisa. Exemplo, na segunda página, aparece dentro da url  o número 49 e as para as páginas seguintes soma-se 48. Mas, para outros termos a segunda página pode ser 50, e as seguintes soma-se 51. Assim, é necessário uma pesquisa anterior para realizar a raspagem. Mas, a estrutura das urls seguem estes padrões.
Segue exemplos abaixo:

página 2 da pesquisa de brinquedo: https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_49_NoIndex_True  
página 3: https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_97_NoIndex_True


Os códigos apresentam breves comentarios para melhor entendimento.


## Patrocinados

Os produtos patrocinados aparecem em cada página, e estão em uma classe do código html diferente dos produtos normais, assim, dentro do bloco de código da Primeira página e de Páginas seguintes a primeira raspagem são nestes produtos e depois nos normais. 

In [100]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [101]:

# função que separa o conteudo da coluna Parcelas (numero de parcelas e valor)
# e realiza a conta do valor total das parcelas

def extrai_valor_parcelas(df):

    # separa o valor de parcela utilizando expressão regular. extrai o valor com virgula que constam após o R$
    df['Valor Parcelas'] = df['Parcelas'].str.extract(r'R\$(\d+,\d+)')
    df['Valor Parcelas'] = df['Valor Parcelas'].fillna(0)
    df['Valor Parcelas'] = df['Valor Parcelas'].apply(lambda x: float(str(x).replace(',', '.')))

    # as parcelas aparecem ao lado da letra x como: '12x'
    # expressão que separa o numeros seguidos por 'x' 
    df['N Parcelas'] = df['Parcelas'].str.extract(r'(\d+)x')
    df['N Parcelas'] = df['N Parcelas'].fillna(0)
    df['N Parcelas'] = df['N Parcelas'].apply(lambda x: int(x))

    df[' Total parcelas'] = df['Valor Parcelas'] * df['N Parcelas']

    return df

# Função Extrai primeira página

In [131]:
# função extrai primeira página

def extrai_primeira_pagina(item):

    base_url = "https://lista.mercadolivre.com.br/"
    
    search_url = f"{base_url}{item}#D[A:{item}]"

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')

    # Lista para armazenar os dados dos produtos
    products_data = []

    products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for i in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
        except:
            product_qt_eval = ''
        
        #frete gratis
        try:
            product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
        except:
            product_fret = ''    

        try: 
            # parcelas
            product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.text
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
        except:
            product_discount = ''


        #link
        product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass

        
    ###############################################

        
        # Adiciona os dados do produto à lista
        products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos
    df_patr = pd.DataFrame(products_data_patr)

    
    #############################################################
    # Produtos nao patrocinados

    for i in range(len(products)):

        
        # Obtém o nome do produto
        product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    

        try: 
            # parcelas
            product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        #link
        product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text
            

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass

        
    ###############################################

        # patrocinado
        try:
        
            patrocinado = products[i].find('div', class_='ui-search-styled-label ui-search-item__pub-label').text.strip()
        except:
            pass
        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'não', 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos
    df = pd.DataFrame(products_data)


    return df_patr, df

In [ ]:
brinquedos-hobbies

# Função extrai paginas seguintes

In [134]:
def extrai_paginas_seguintes(item, categorias: list):

    # DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
    all_products_df = pd.DataFrame()
    all_products_df_patr = pd.DataFrame()

    for i in range(1, 5):
        n = 49 

        if len(categorias) == 1:
            search_url = f'https://lista.mercadolivre.com.br/{categorias[0]}/{item}{n}_NoIndex_True'

        else:
            search_url = f'https://lista.mercadolivre.com.br/{categorias[0]}/{categorias[1]}/{item}_Desde_49_NoIndex_True'

        # Realiza a requisição HTTP
        response = requests.get(search_url)

        # Verifica se a requisição foi bem-sucedida
        if response.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")
            continue

        # Parsing do HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lista para armazenar os dados dos produtos desta iteração
        products_data = []

        # Encontra os elementos que contêm as informações do produto
        products = soup.find_all(class_='ui-search-result__content-wrapper')



        
        all_products_data_patr = []


        # produtos patrocinados
        products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
        products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

        for x in range(len(products_patr)):

            
            # Obtém o nome do produto
            product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
            
            # Obtém o preço do produto
            product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

            try:
            # avaliação
                product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
            except:
                pass

            try:
            # qtde avaliações
                product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
            except:
                pass
            
            #frete gratis
            try:
                product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
            except:
                pass    

            try: 
                # parcelas
                product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

                # Extrai o texto do elemento
                product_parc = product_parc.get_text(strip=True)
            except:
                pass

            # sem juros    
            try:
                product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
                product_juros = product_juros.find(text='juros')

            except:
                pass
                # desconto
            try:
                product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
            except:
                pass


            #link
            product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

            # Extrai o atributo href do elemento
            product_link = product_link['href']


            # DADOS LINK

            response2 = requests.get(product_link)

            # Verifica se a requisição foi bem-sucedida
            if response2.status_code != 200:
                print(f"Erro ao acessar a URL: {search_url}")

            # Parsing do HTML
            soup2 = BeautifulSoup(response2.text, 'html.parser')


            # Nome vendedor
            try:
                #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
                #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
                #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
                info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

                # Extrai o vendedor 
                seller = info_element.text

            except:
                pass
            # posição de venda na categoria, se possui
            try:

                #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
                #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
                position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
                position = position[1].text

            except:
                pass


            try:
            # Novo/usado e qtde vendida
                #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
                situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


                # Separa as informações 'Novo' e '+500 vendidos'
                info_parts = situation.text.split('|')

                # Remove os espaços em branco ao redor de cada parte
                info_parts = [part.strip() for part in info_parts]

                product_situation = info_parts[0]
                product_sells = info_parts[1]
            except:
                pass

                # Quant. de fotos
            try:
                # qtde fotos
                #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
                #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

                #images= images.find('div',class_ = 'ui-pdp-gallery__column')
                images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

                images = len(images)
            except:
                pass

            
        ###############################################

            
            # Adiciona os dados do produto à lista
            all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                                'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                                'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                                'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 'Quant. fotos': images})

        # Cria um DataFrame com os dados dos produtos

        patr_temp_df = pd.DataFrame(all_products_data_patr)
        all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
        
        









        for product in products:
            # Obtém o nome do produto
            product_name = product.find('h2', class_='ui-search-item__title').text.strip()

            # Obtém o preço do produto
            product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


            try:
        # avaliação
                product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
            except:
                pass

            try:
            # qtde avaliações
                product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
            except:
                pass
            
            #frete gratis
            try:
                product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
            except:
                pass    


            try: 
            # parcelas
                product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

                # Extrai o texto do elemento
                product_parc = product_parc.get_text(strip=True)
            except:
                pass

            # sem juros    
            try:
                product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
                product_juros = product_juros.find(text='juros')

            except:
                pass
            # desconto
            try:
                product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
            except:
                product_discount = ''


            

            #link
            product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

            # Extrai o atributo href do elemento
            product_link = product_link['href']


            # DADOS LINK

            response2 = requests.get(product_link)

            # Verifica se a requisição foi bem-sucedida
            if response2.status_code != 200:
                print(f"Erro ao acessar a URL: {search_url}")

            # Parsing do HTML
            soup2 = BeautifulSoup(response2.text, 'html.parser')

            # Vendedor
            try:
                #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
                #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
                #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
                info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

                # Extrai o texto do elemento
                seller = info_element
            except:
                pass

            
            # posição de venda na categoria, se possui
            try:

                #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
                #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
                #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
                position = soup2[1].text

            except:
                pass

                # Novo/usado e qtde vendida
            try:
            
                #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
                situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


                # Separa as informações 'Novo' e '+500 vendidos'
                info_parts = situation.text.split('|')

                # Remove os espaços em branco ao redor de cada parte
                info_parts = [part.strip() for part in info_parts]

                product_situation = info_parts[0]
                product_sells = info_parts[1]
            except:
                pass

                # Quant. de fotos
            try:
                
                #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
                #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

                #images= images.find('div',class_ = 'ui-pdp-gallery__column')
                images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

                images = len(images)
            except:
                pass


    ################################################

            patrocinado_tag = product.find('label', class_='ui-search-styled-label.ui-search-item__pub-label')
            patrocinado = patrocinado_tag.text.strip() if patrocinado_tag else "Não disponível"

            # Adiciona os dados do produto à lista
            products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc,
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 'Quant. fotos': images})

        n = n + 48
        # Cria um DataFrame temporário com os dados dos produtos desta iteração
        temp_df = pd.DataFrame(products_data)

        # Concatena o DataFrame temporário ao DataFrame principal
        all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)

        return all_products_df_patr, all_products_df

# Brinquedos

### Primeira Página

In [102]:
base_url = "https://lista.mercadolivre.com.br/"
item = 'brinquedo'


search_url = f"{base_url}{item}#D[A:{item}]"

# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML 
soup = BeautifulSoup(response.text, 'html.parser')

# Classe html dos produtos normais
# Encontra os elementos que contêm as informações do produto
# todos os produtos tem essa classe, então usa-se o find all para trazer todos
products = soup.find_all(class_='ui-search-result__content-wrapper')


# Lista para armazenar os dados dos produtos patrocinados
products_data_patr = []

# Lista para armazenar os dados dos produtos normais
products_data = []


###########################################################################
# Classe html dos produtos patrocinados
# Encontra os elementos que contêm as informações do produto
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')



for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href que está o link do produto
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass


        # Quant. de fotos
    try:
        
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


#CATEGORIAS
##########################################


    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    
    
    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)






#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])

    
###############################################

    
    # Adiciona os dados do produto à lista
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)


### Páginas seguintes

In [103]:
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_{n}_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        # Obtém o preço do produto
        product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})



####    # Incrementa a posição conforme a págin seguinte
    
    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)


Erro ao acessar a URL: https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_49_NoIndex_True


In [104]:

# juntando os dataframes
df_mercado_brinquedo = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')

In [105]:
df_mercado_brinquedo = extrai_valor_parcelas(df_mercado_brinquedo)
df_mercado_brinquedo.drop(['Parcelas'], axis = 1, inplace = True)
# convertendo o preço para número decimal
df_mercado_brinquedo['Preço'] = df_mercado_brinquedo['Preço'].apply(lambda x: float(x.replace('R$', '').replace(',', '.')))


In [106]:
caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\brinquedos_mercadolivre.xlsx'
df_mercado_brinquedo.to_excel(caminho, index=False)

# Escritório

### Primeira Página

In [107]:
base_url = "https://lista.mercadolivre.com.br/"
item = 'escritorio'
search_url = f"{base_url}{item}#D[A:{item}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os elementos que contêm as informações do produto
products = soup.find_all(class_='ui-search-result__content-wrapper')

# Lista para armazenar os dados dos produtos
products_data = []

products_data_patr = []


# produtos patrocinados
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')







for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass
#CATEGORIAS
##########################################


           

    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)


#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])

    
###############################################

    

    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)

### Paginas Seguintes

In [108]:
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/casa-moveis-decoracao/escritorio_Desde_{n}_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])

    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        # Obtém o preço do produto
        product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])

    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)

In [109]:
# juntando os dataframes
df_mercado_escritorio = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')

### Tratamento

In [110]:
df_mercado_escritorio = extrai_valor_parcelas(df_mercado_escritorio)
df_mercado_escritorio.drop(['Parcelas'], axis = 1, inplace = True)
# removendo os parenteses das avaliações e convertendo o número para inteiro
df_mercado_escritorio['Qtde avaliações:'] = df_mercado_escritorio['Qtde avaliações:'].apply(lambda x: int(x.replace('(', '').replace(')','')))
# convertendo o preço para número decimal
df_mercado_escritorio['Preço'] = df_mercado_escritorio['Preço'].apply(lambda x: float(x.replace('R$', '').replace(',', '.')))


In [111]:
caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\escritorio_mercadolivre.xlsx'
df_mercado_escritorio.to_excel(caminho, index=False)

# Mochila

### Primeira Página

In [112]:
base_url = "https://lista.mercadolivre.com.br/"
item = 'mochila'
search_url = f"{base_url}{item}#D[A:{item}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os elementos que contêm as informações do produto
products = soup.find_all(class_='ui-search-result__content-wrapper')

# Lista para armazenar os dados dos produtos
products_data = []

products_data_patr = []


# produtos patrocinados
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')







for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass
#CATEGORIAS
##########################################


           

    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)


#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])

    
###############################################

    
    # Adiciona os dados do produto à lista
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)


### Paginas Seguintes

In [113]:

# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/calcados-roupas-bolsas/malas-bolsas/mochilas/mochila_Desde_{n}_AGE*GROUP_6725189_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])

    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        # Obtém o preço do produto
        product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])

    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)

In [114]:
df_mercado_mochila = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
df_mercado_mochila = extrai_valor_parcelas(df_mercado_mochila)
df_mercado_mochila.drop(['Parcelas'], axis = 1, inplace = True)
# removendo os parenteses das avaliações e convertendo o número para inteiro
df_mercado_mochila['Qtde avaliações:'] = df_mercado_mochila['Qtde avaliações:'].apply(lambda x: int(x.replace('(', '').replace(')','')))
# convertendo o preço para número decimal
df_mercado_mochila['Preço'] = df_mercado_mochila['Preço'].apply(lambda x: float(x.replace('R$', '').replace(',', '.')))

caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\mochila_mercadolivre.xlsx'
df_mercado_mochila.to_excel(caminho, index=False)

In [115]:
# Pesquisa para mais de uma palavra
def formatar_para_url2(string):
    # Substitui os espaços por "%20"
    return string.replace(" ", "%20")

def formatar_para_url1(string):
    # Substitui os espaços por "%20"
    return string.replace(" ", "-")

# Papelaria

### Primeira Página

In [116]:
base_url = "https://lista.mercadolivre.com.br/"
item = 'papelaria'
search_url = f"{base_url}{item}#D[A:{item}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os elementos que contêm as informações do produto
products = soup.find_all(class_='ui-search-result__content-wrapper')

# Lista para armazenar os dados dos produtos
products_data = []

products_data_patr = []


# produtos patrocinados
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')







for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass
#CATEGORIAS
##########################################


           

    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    

    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)


#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    
###############################################

    
    
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)

### Páginas Seguintes

In [117]:
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/arte-papelaria-armarinho/materiais-escolares/papelaria_Desde_{n}_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        # Obtém o preço do produto
        product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])

    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)

### Tratamento

In [118]:
df_mercado_papelaria = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
df_mercado_papelaria = extrai_valor_parcelas(df_mercado_papelaria)
df_mercado_papelaria.drop(['Parcelas'], axis = 1, inplace = True)
# removendo os parenteses das avaliações e convertendo o número para inteiro
#df_mercado_papelaria['Qtde avaliações:'] = df_mercado_papelaria['Qtde avaliações:'].apply(lambda x: int(x.replace('(', '').replace(')','')))
# convertendo o preço para número decimal
df_mercado_papelaria['Preço'] = df_mercado_papelaria['Preço'].apply(lambda x: float(x.replace('R$', '').replace(',', '.')))

caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\papelaria_mercadolivre.xlsx'
df_mercado_papelaria.to_excel(caminho, index=False)

# PC

## Primeira Página

In [119]:
base_url = "https://lista.mercadolivre.com.br/"
item = 'pc'
search_url = f"{base_url}{item}#D[A:{item}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os elementos que contêm as informações do produto
products = soup.find_all(class_='ui-search-result__content-wrapper')

# Lista para armazenar os dados dos produtos
products_data = []

products_data_patr = []


# produtos patrocinados
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')







for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass
#CATEGORIAS
##########################################


           

    #data_top = soup2.find('div',class_='ui-pdp')
    #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

    #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

    #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
    #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
    #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    

    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)


#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    try:
        #data_top = soup2.find('div',class_='ui-pdp')
        #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

        #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

        #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
        #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
        #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    except:
        categories_list = []

    
###############################################

    
    # Adiciona os dados do produto à lista
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)


## Paginas seguintes

In [120]:
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/informatica/pc-mesa/computadores/pc_Desde_{n}_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        try:
            #data_top = soup2.find('div',class_='ui-pdp')
            #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
            #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
            #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

            data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

            categories_list = []
            

            for cat in data_top:

                categories_list.append(cat.text)
        except:
             categories_list = []

    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        # Obtém o preço do produto
        product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        try:
            #data_top = soup2.find('div',class_='ui-pdp')
            #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
            #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
            #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

            data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

            categories_list = []
            

            for cat in data_top:

                categories_list.append(cat.text)
        except:
             categories_list = []

    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)

## Tratamento

In [121]:
df_mercado_pc = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
df_mercado_pc = extrai_valor_parcelas(df_mercado_pc)
df_mercado_pc.drop(['Parcelas'], axis = 1, inplace = True)
# removendo os parenteses das avaliações e convertendo o número para inteiro
df_mercado_pc['Qtde avaliações:'] = df_mercado_pc['Qtde avaliações:'].apply(lambda x: x if x == '' else int(x.replace('(', '').replace(')','')))
# convertendo o preço para número decimal
df_mercado_pc['Preço'] = df_mercado_pc['Preço'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))

caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\pc_mercadolivre.xlsx'
df_mercado_pc.to_excel(caminho, index=False)

# Notebook

## Primeira Página

In [122]:
base_url = "https://lista.mercadolivre.com.br/"
item = 'notebook'
search_url = f"{base_url}{item}#D[A:{item}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os elementos que contêm as informações do produto
products = soup.find_all(class_='ui-search-result__content-wrapper')

# Lista para armazenar os dados dos produtos
products_data = []

products_data_patr = []


# produtos patrocinados
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')







for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        product_juros = ''
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        product_discount = ''


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass
#CATEGORIAS
##########################################


           

    #data_top = soup2.find('div',class_='ui-pdp')
    #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

    #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

    #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
    #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
    #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    

    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)


#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    try:
        #data_top = soup2.find('div',class_='ui-pdp')
        #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

        #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

        #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
        #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
        #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    except:
        categories_list = []

    
###############################################

    
    # Adiciona os dados do produto à lista
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)


## Páginas seguintes

In [126]:
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/informatica/portateis-acessorios/notebooks/notebook_Desde_{n}_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        try:
            #data_top = soup2.find('div',class_='ui-pdp')
            #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
            #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
            #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

            data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

            categories_list = []
            

            for cat in data_top:

                categories_list.append(cat.text)
        except:
             categories_list = []

    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto

        try:
            product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()
        except:
            product_price = product.find('span', class_='andes-money-amount__fraction').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        try:
            #data_top = soup2.find('div',class_='ui-pdp')
            #data_top = soup2.find('div',class_='ui-pdp-container ui-pdp-container--top')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__row ui-pdp-group-header-breadcrumb')

            #data_top = data_top.find('div', class_ = 'ui-pdp-container__col col-2')
            #data_top = data_top.find('div', class_ = 'ui-pdp-breadcrumb')
            #data_top = data_top.find('ol', class_ = 'andes-breadcrumb')

            data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

            categories_list = []
            

            for cat in data_top:

                categories_list.append(cat.text)
        except:
             categories_list = []

    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)

Notebook Ideapad 1 I3 8gb 256gb W11 Home 15.6 82vy000sbr
2.480
MacBook Air M2 2022 midnight 13.6", Apple M2  8GB de RAM 256GB SSD, Apple M2 10-Core GPU 2560x1664px macOS
8.199
Apple Macbook Air (13 polegadas, 2020, Chip M1, 256 GB de SSD, 8 GB de RAM) - Cinza-espacial
5.943
Notebook Dell Latitude E5450 Core I5  16 Gb Ram  480 Gb Ssd
1.799
Notebook Aspire 5 A515-57-58w1 I5 Linux 8gb 256gb 15,6' Fhd
3.559
Notebook Loq I5-12450h 16gb 512gb Ssd Rtx 3050 15.6 Fhd W11
5.799
Notebook Gamer Acer Nitro 5 17.3 144hz I5 512gb 8gb Rtx 3050 Cor Preto
5.942
Notebook Lenovo IdeaPad 1i Intel Core i7-1255U 16GB 512GB SSD Windows 11 Home 15.6" 82VY000NBR
3.679
Notebook Elitebook Hp 840 I5 6300u 16gb Ssd 256gb Win 11
1.849
Notebook Multi Ultra Celeron N4020c 4gb 128gb W11 15,6'' Cinza - Ub261
2.256
Notebook Acer Aspire A315-24p-r06b 8gb 512gb W11 15.6  Prata Cor Prateado
2.699
Notebook Lenovo IdeaPad 1i Intel Core i3-1215U 4GB 256GB SSD Windows 11 Home 14" 83AF0000BR
1.919
Notebook Galaxy Book Go Snapdra

## Tratamento

In [ ]:
df_mercado_notebbok = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
df_mercado_notebbok = extrai_valor_parcelas(df_mercado_notebbok)
df_mercado_notebbok.drop(['Parcelas'], axis = 1, inplace = True)
# removendo os parenteses das avaliações e convertendo o número para inteiro
df_mercado_notebbok['Qtde avaliações:'] = df_mercado_notebbok['Qtde avaliações:'].apply(lambda x: x if x == '' else int(x.replace('(', '').replace(')','')))
# convertendo o preço para número decimal
df_mercado_notebbok['Preço'] = df_mercado_notebbok['Preço'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))

caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\notebook_mercadolivre.xlsx'
df_mercado_notebbok.to_excel(caminho, index=False)

# Meias

In [ ]:
## Primeira Página
base_url = "https://lista.mercadolivre.com.br/"
item = 'meias'
search_url = f"{base_url}{item}#D[A:{item}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os elementos que contêm as informações do produto
products = soup.find_all(class_='ui-search-result__content-wrapper')

# Lista para armazenar os dados dos produtos
products_data = []

products_data_patr = []


# produtos patrocinados
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')







for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        product_juros = ''
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        product_discount = ''


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass
#CATEGORIAS
##########################################


           

    

    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    

    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)


#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])

    
###############################################

    
    # Adiciona os dados do produto à lista
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)

## Páginas seguintes
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/meias_Desde_{n}_AGE*GROUP_6725189_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        # Obtém o preço do produto
        product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)
## Tratamento
df_mercado_meias = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
df_mercado_meias = extrai_valor_parcelas(df_mercado_meias)
df_mercado_meias.drop(['Parcelas'], axis = 1, inplace = True)
# removendo os parenteses das avaliações e convertendo o número para inteiro
df_mercado_meias['Qtde avaliações:'] = df_mercado_meias['Qtde avaliações:'].apply(lambda x: x if x == '' else int(x.replace('(', '').replace(')','')))
# convertendo o preço para número decimal
df_mercado_meias['Preço'] = df_mercado_meias['Preço'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))

caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\meias_mercadolivre.xlsx'
df_mercado_meias.to_excel(caminho, index=False)

Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.m

# Kit de meias

In [ ]:
## Primeira Página
base_url = "https://lista.mercadolivre.com.br/"
item = 'kit de meias'

search_url = f"{base_url}{item.replace(' ', '-')}#D[A:{item.replace(' ', '%20')}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os elementos que contêm as informações do produto
products = soup.find_all(class_='ui-search-result__content-wrapper')

# Lista para armazenar os dados dos produtos
products_data = []

products_data_patr = []


# produtos patrocinados
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
products_patr= products_patr.find_all('li', class_='ui-search-layout__item')







for i in range(len(products_patr)):

    
    # Obtém o nome do produto
    product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number')
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
    except:
        product_qt_eval = ''
    
    #frete gratis
    try:
        product_fret = products_patr[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.text
    except:
        pass

    # sem juros    
    try:
        product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        product_juros = ''
        # desconto
    try:
        product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        product_discount = ''


    #link
    product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass
#CATEGORIAS
##########################################


           

    

    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    

    
###############################################

    
    # Adiciona os dados do produto à lista
    products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df_patr = pd.DataFrame(products_data_patr)


#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        pass    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        pass


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    # posição de venda na categoria, se possui
    try:

        #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
        position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text

    except:
        pass


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass


    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])
    
###############################################

    
    # Adiciona os dados do produto à lista
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)








## Páginas seguintes
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    n = 49 
    search_url = f'https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/kit-de-meias_Desde_{49}_NoIndex_True'

    # Realiza a requisição HTTP
    response = requests.get(search_url)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    products_patr= products_patr.find_all('li', class_='ui-search-layout__item')

    for x in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

        try:
        # avaliação
            product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = products_patr[x].find('span', class_='ui-pb-highlight')
        except:
            pass    

        try: 
            # parcelas
            product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[x].find('span', class_='ui-search-price__discount')
        except:
            pass


        #link
        product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            position = soup2.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text

        except:
            pass


        try:
        # Novo/usado e qtde vendida
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            # qtde fotos
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass
        



        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])

    
###############################################

    
          
    

        
        # Adiciona os dados do produto à lista
        all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    # Cria um DataFrame com os dados dos produtos

    patr_temp_df = pd.DataFrame(all_products_data_patr)
    all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
    










    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        # Obtém o preço do produto
        product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            pass    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            pass


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
        try:

            #position = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            #position = position.find('div',class_ = 'ui-pdp-promotions-pill mt-10 ui-pdp-highlights')
            #position = position.find_all('a',class_ = 'ui-pdp-promotions-pill-label__target')
            position = soup2[1].text

        except:
            pass

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass


#####################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])

    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': 'sim', 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images})

    n = n + 48
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)
## Tratamento
df_mercado_kitmeias = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
df_mercado_kitmeias = extrai_valor_parcelas(df_mercado_kitmeias)
df_mercado_kitmeias.drop(['Parcelas'], axis = 1, inplace = True)
# removendo os parenteses das avaliações e convertendo o número para inteiro
df_mercado_kitmeias['Qtde avaliações:'] = df_mercado_kitmeias['Qtde avaliações:'].apply(lambda x: x if x == '' else int(x.replace('(', '').replace(')','')))
# convertendo o preço para número decimal
df_mercado_kitmeias['Preço'] = df_mercado_kitmeias['Preço'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))

caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\kitmeias_mercadolivre.xlsx'
df_mercado_kitmeias.to_excel(caminho, index=False)

Erro ao acessar a URL: https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/kit-de-meias_Desde_49_NoIndex_True
Erro ao acessar a URL: https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/kit-de-meias_Desde_49_NoIndex_True
Erro ao acessar a URL: https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/kit-de-meias_Desde_49_NoIndex_True
Erro ao acessar a URL: https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/kit-de-meias_Desde_49_NoIndex_True
Erro ao acessar a URL: https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/kit-de-meias_Desde_49_NoIndex_True
Erro ao acessar a URL: https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/mei

# Tratamento Geral

In [ ]:
df_mercado_brinquedo['Categoria'] = 'Brinquedo'
df_mercado_escritorio['Categoria'] = "Escritório"
df_mercado_mochila['Categoria'] = 'Mochila'
df_mercado_papelaria['Categoria'] = 'Papelaria'
df_mercado_pc['Categoria'] = 'PC'
df_mercado_notebbok['Categoria'] = 'Notebook'
df_mercado_meias['Categoria'] = 'Meias'
df_mercado_kitmeias['Categoria'] = 'Kit Meias'

df_geral = pd.concat([df_mercado_brinquedo, df_mercado_escritorio, df_mercado_mochila, 
                      df_mercado_papelaria, df_mercado_pc, df_mercado_notebbok,df_mercado_meias, df_mercado_kitmeias ], join = 'outer')


df_geral['Categoria0'] = 0
df_geral['Categoria1'] = 0
df_geral['Categoria2'] = 0
df_geral['Categoria3'] = 0
df_geral['Categoria4'] = 0
df_geral['Categoria5'] = 0
df_geral['Categoria6'] = 0

for i in range(len(df_geral['Categorias'])):

    for x in range(len(df_geral['Categorias'].iloc[i])):
        df_geral[f'Categoria{x}'].iloc[i] = df_geral['Categorias'].iloc[i][x]

        
        

C:\Users\prpau\AppData\Local\Temp\ipykernel_11020\656822717.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral[f'Categoria{x}'].iloc[i] = df_geral['Categorias'].iloc[i][x]


In [ ]:
for i in range(len(df_geral['Categorias'])):

        for x in range(0,7):
            if df_geral[f'Categoria{x}'].iloc[i] == 0:
                df_geral[f'Categoria{x}'].iloc[i] = df_geral[f'Categoria{x-1}'].iloc[i]

KeyError: 'Categoria-1'

In [ ]:
caminho = 'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\mercadolivre_atualizado.xlsx'
df_geral.to_excel(caminho)